## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [RxNorm sentence entity resolver](https://aws.amazon.com/marketplace/pp/prodview-wfezzkzs6y57k)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for RxNorm Concept Unique Identifier (RxCUI) Sentence Entity Resolver

- **Model**: `rxnorm_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts `DRUG` entities from clinical text and maps them to their corresponding RxNorm Concept Unique Identifier (RxCUI) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "rxnorm-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    "She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",
    "The patient was advised to take Omeprazole 20 mg daily before breakfast for acid reflux.",
]


sample_text = """He was seen by the endocrinology service and she was discharged on 50 mg of eltrombopag oral at night, 5 mg amlodipine with meals, and metformin 1000 mg two times a day."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,concept_class_id,all_codes,concept_name_detailed,all_resolutions,all_score
0,45,91,she was discharged on 50 mg of eltrombopag oral,DRUG,0.6516222,825426,eltrombopag 50 mg,0.808447,"[Clinical Drug Comp, Branded Drug Comp, Branded Drug, Clinical Drug, Clinical Drug]","[825426, 825428, 825429, 825427, 825427]","[eltrombopag 50 MG[eltrombopag 50 MG], eltrombopag 50 MG [Promacta][eltrombopag 50 MG [Promacta]], eltrombopag 50 MG Oral Tablet [Promacta][eltrombopag 50 MG Oral Tablet [Promacta]], eltrombopag 50 MG Oral Tablet[eltrombopag 50 MG Oral Tablet], eltrombopag (as eltrombopag olamine) 50 MG Oral Tablet [eltrombopag 50 MG Oral Tablet]]","[eltrombopag 50 mg, eltrombopag 50 mg [promacta], eltrombopag 50 mg oral tablet [promacta], eltrombopag 50 mg oral tablet, eltrombopag (as eltrombopag olamine) 50 mg oral tablet]","[0.8084468841552734, 0.8038963675498962, 0.7828861474990845, 0.7770534157752991, 0.765609622001648]"
1,103,117,5 mg amlodipine,DRUG,0.5661333,329528,amlodipine 5 mg,0.987805,"[Clinical Drug Comp, Clinical Drug Comp, Clinical Drug Comp, Branded Drug Comp, Clinical Drug]","[329528, 329527, 2377370, 572722, 197361]","[amlodipine 5 MG[amlodipine 5 MG], amlodipine 2.5 MG[amlodipine 2.5 MG], levamlodipine 5 MG[levamlodipine 5 MG], amlodipine 5 MG [Norvasc][amlodipine 5 MG [Norvasc]], amlodipine 5 MG Oral Tablet[amlodipine 5 MG Oral Tablet]]","[amlodipine 5 mg, amlodipine 2.5 mg, levamlodipine 5 mg, amlodipine 5 mg [norvasc], amlodipine 5 mg oral tablet]","[0.9878053665161133, 0.9416172504425049, 0.9309700727462769, 0.9125789403915405, 0.9114688634872437]"
2,135,151,metformin 1000 mg,DRUG,0.6566333,316255,metformin 1000 mg,1.000000,"[Clinical Drug Comp, Clinical Drug Comp, Clinical Drug Comp, Clinical Drug Comp, Branded Drug Comp]","[316255, 316255, 860995, 330861, 860997]","[metformin 1000 MG[metformin 1000 MG], metformin 1000 MG, metformin hydrochloride 1000 MG[metformin hydrochloride 1000 MG], metformin 250 MG[metformin 250 MG], metformin hydrochloride 1000 MG [Fortamet][metformin hydrochloride 1000 MG [Fortamet]]]","[metformin 1000 mg, metformin hcl 1000 mg, metformin hydrochloride 1000 mg, metformin 250 mg, metformin hydrochloride 1000 mg [fortamet]]","[1.0, 0.9527718424797058, 0.9353886842727661, 0.9127198457717896, 0.9050541520118713]"


### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 45, "end": 91, "ner_chunk": "she was discharged on 50 mg of eltrombopag oral", "ner_label": "DRUG", "ner_confidence": "0.6516222", "concept_code": "825426", "resolution": "eltrombopag 50 mg", "score": 0.8084468841552734, "concept_class_id": ["Clinical Drug Comp", "Branded Drug Comp", "Branded Drug", "Clinical Drug", "Clinical Drug"], "all_codes": ["825426", "825428", "825429", "825427", "825427"], "concept_name_detailed": ["eltrombopag 50 MG[eltrombopag 50 MG]", "eltrombopag 50 MG [Promacta][eltrombopag 50 MG [Promacta]]", "eltrombopag 50 MG Oral Tablet [Promacta][eltrombopag 50 MG Oral Tablet [Promacta]]", "eltrombopag 50 MG Oral Tablet[eltrombopag 50 MG Oral Tablet]", "eltrombopag (as eltrombopag olamine) 50 MG Oral Tablet [eltrombopag 50 MG Oral Tablet]"], "all_resolutions": ["eltrombopag 50 mg", "eltrombopag 50 mg [promacta]", "eltrombopag 50 mg oral tablet [promacta]", "eltrombopag 50 mg oral tablet", "eltrombopag (as eltrombopag olamine) 50 mg oral tabl

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'begin': 13,
    'end': 27,
    'ner_chunk': 'folic acid 1 mg',
    'ner_label': 'DRUG',
    'ner_confidence': '0.67852503',
    'concept_code': '315966',
    'resolution': 'folic acid 1 mg',
    'score': 1.0000001192092896,
    'concept_class_id': ['Clinical Drug Comp',
     'Clinical Drug Comp',
     'Clinical Drug Comp',
     'Branded Drug Comp',
     'Clinical Drug Comp'],
    'all_codes': ['315966', '1087277', '334757', '542061', '336541'],
    'concept_name_detailed': ['folic acid 1 MG[folic acid 1 MG]',
     'folic acid 1.1 MG[folic acid 1.1 MG]',
     'folic acid 0.1 MG[folic acid 0.1 MG]',
     'folic acid 1 MG [Folicet][folic acid 1 MG [Folicet]]',
     'folic acid 1 MG/ML[folic acid 1 MG/ML]'],
    'all_resolutions': ['folic acid 1 mg',
     'folic acid 1.1 mg',
     'folic acid 0.1 mg',
     'folic acid 1 mg [folicet]',
     'folic acid 1 mg/ml'],
    'all_score': [1.0000001192092896,
     0.9873265027999878,
     0.9669881463050842,
     0.96443587541580

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"begin": 13, "end": 27, "ner_chunk": "folic acid 1 mg", "ner_label": "DRUG", "ner_confidence": "0.67852503", "concept_code": "315966", "resolution": "folic acid 1 mg", "score": 1.0000001192092896, "concept_class_id": ["Clinical Drug Comp", "Clinical Drug Comp", "Clinical Drug Comp", "Branded Drug Comp", "Clinical Drug Comp"], "all_codes": ["315966", "1087277", "334757", "542061", "336541"], "concept_name_detailed": ["folic acid 1 MG[folic acid 1 MG]", "folic acid 1.1 MG[folic acid 1.1 MG]", "folic acid 0.1 MG[folic acid 0.1 MG]", "folic acid 1 MG [Folicet][folic acid 1 MG [Folicet]]", "folic acid 1 MG/ML[folic acid 1 MG/ML]"], "all_resolutions": ["folic acid 1 mg", "folic acid 1.1 mg", "folic acid 0.1 mg", "folic acid 1 mg [folicet]", "folic acid 1 mg/ml"], "all_score": [1.0000001192092896, 0.9873265027999878, 0.9669881463050842, 0.964435875415802, 0.9637104272842407]}, {"begin": 37, "end": 56, "ner_chunk": "levothyroxine 0.1 mg", "ner_label": "DRUG", "ner_confidence"

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

